In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
from keras.preprocessing.image import ImageDataGenerator,load_img
import cv2
from random import shuffle
from shutil import copyfile,rmtree
import json
from imgaug import augmenters as iaa
import numpy as np
from PIL import Image

class ImagePrep:
    
    def getTrainTest(self,directory,extention):
        files = []
        for file in os.listdir(directory):
            if file.endswith(extention):
                files.append(file)
        shuffle(files)
        splt = int(len(files)*0.8)
        x_train_files = files[0:splt]
        x_test_files = files[splt:-1]
        
        json_data = open(directory +'/calories_json.json',mode='r').read()
        y_map = json.loads(json_data)
        
        x_train = [load_img(directory+"/"+x, target_size=(224,224,3)) for x in x_train_files]
        y_train = [y_map[x] for x in x_train_files]
        x_test = [load_img(directory+"/"+x, target_size=(224,224,3)) for x in x_test_files]
        y_test = [y_map[x] for x in x_test_files]
        
        return x_train,y_train,x_test,y_test
    
    def augmentImages(self,x,y):
        x_train_aug = []
        y_train_aug = []
        
        for i in range(len(x)):
            x_train_aug.append(x[i])
            y_train_aug.append(y[i])
            
            #transformation one - adding random black dots in the image
            t = iaa.CoarseDropout(0.1, size_percent=0.2).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation two - simple rotation
            t = iaa.Affine(rotate=(-25, 25)).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
        
            #transformation three - increasing the contrast of the objects in the image
            t = iaa.GammaContrast(1.5).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation four - mirror effect
            t = iaa.Fliplr(p = 1.0).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation five - blurring effect
            t = iaa.GaussianBlur((0, 3.0)).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation six - cropping and padding, expand, squish, move effect
            t = iaa.CropAndPad(percent=(-0.25, 0.25)).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
            #transformation seven - it randomly moves pixels around within a given bound
            t = iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25).augment_images([np.array(x[i])])
            x_train_aug.append(Image.fromarray(np.uint8(t[0])))
            y_train_aug.append(y[i])
            
        return x_train_aug,y_train_aug

Using TensorFlow backend.


In [0]:
obj = ImagePrep()
x_train,y_train,x_test,y_test = obj.getTrainTest("drive/My Drive/subway",'.png')
x_train_aug,y_train_aug = obj.augmentImages(x_train,y_train)

x_train_aug_np = np.array([np.array(x) for x in x_train_aug])
y_train_aug = np.array(y_train_aug).astype('int')

In [0]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
from keras.losses import mean_squared_error as mse
from keras import layers,models
from keras.applications.resnet50 import ResNet50

class Models:

    def cnn(
        self,
        width,
        height,
        depth,
        filters=(16, 32, 64),
        regress=False,
        ):

        # initialize the input shape and channel dimension, assuming
        # TensorFlow/channels-last ordering

        inputShape = (height, width, depth)
        chanDim = -1

        # define the model input

        inputs = Input(shape=inputShape)

        # loop over the number of filters

        for (i, f) in enumerate(filters):

            # if this is the first CONV layer then set the input
            # appropriately

            if i == 0:
                x = inputs

            # CONV => RELU => BN => POOL

            x = Conv2D(f, (3, 3), padding='same')(x)
            x = Activation('relu')(x)
            x = BatchNormalization(axis=chanDim)(x)
            x = MaxPooling2D(pool_size=(2, 2))(x)

            # flatten the volume, then FC => RELU => BN => DROPOUT

        x = Flatten()(x)
        x = Dense(16)(x)
        x = Activation('relu')(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Dropout(0.5)(x)

        # apply another FC layer, this one to match the number of nodes
        # coming out of the MLP

        x = Dense(4)(x)
        x = Activation('relu')(x)

        # check to see if the regression node should be added

        if regress:
            x = Dense(1, activation='linear')(x)

        # construct the CNN

        model = Model(inputs, x)

        # return the CNN

        return model

    def resnet(self, freezing):
        resnet50_model = ResNet50(include_top=True, weights='imagenet')

        # in this model we are freezing layers(3,116) inclusive freezing

        if freezing:
            times = 0
        for i in range(3, 200, 1):
            if 'BatchNormalization' \
                in str(type(resnet50_model.layers[i])) and times == 40:
                break
            else:
                if 'BatchNormalization' \
                    in str(type(resnet50_model.layers[i])):
                    times += 1
            resnet50_model.layers[i].trainable = False
        regression_model = models.Sequential()
        resnet50_model.layers[176] = layers.Dense(512, activation='relu'
                )
        regression_model.add(resnet50_model)
        regression_model.add(layers.Dense(256, activation='sigmoid'))
        regression_model.add(layers.Dense(128, activation='relu'))
        regression_model.add(layers.Dense(1, activation='linear'))
        regression_model.compile(optimizer='sgd',
                                 loss='mean_squared_error',
                                 metrics=['accuracy'])
        return regression_model


In [5]:
models = Models()
cnn_model = models.cnn(224,224,3,regress=True)













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
from keras.optimizers import Adam
opt = Adam(lr=1e-3, decay=1e-3 / 200)
cnn_model.compile(loss="mean_absolute_percentage_error", optimizer=opt)

In [7]:
cnn_model.fit(x_train_aug_np, y_train_aug, epochs=30, batch_size=8)



Epoch 1/30
960/960 [==============================] - 8s 8ms/step - loss: 99.7085
Epoch 2/30
960/960 [==============================] - 4s 4ms/step - loss: 98.8013
Epoch 3/30
960/960 [==============================] - 4s 4ms/step - loss: 97.0321
Epoch 4/30
960/960 [==============================] - 4s 4ms/step - loss: 93.9480
Epoch 5/30
960/960 [==============================] - 4s 4ms/step - loss: 88.3963
Epoch 6/30
960/960 [==============================] - 4s 4ms/step - loss: 81.4801
Epoch 7/30
960/960 [==============================] - 4s 4ms/step - loss: 73.7836
Epoch 8/30
960/960 [==============================] - 4s 4ms/step - loss: 67.5891
Epoch 9/30
960/960 [==============================] - 4s 4ms/step - loss: 62.1036
Epoch 10/30
960/960 [==============================] - 4s 4ms/step - loss: 57.0451
Epoch 11/30
960/960 [==============================] - 4s 4ms/step - loss: 56.3206
Epoch 12/30
960/960 [==============================] - 4s 4ms/step - loss: 53.6574
Epoch 13/30

In [0]:
x_test_np = np.array([np.array(x) for x in x_test])
y_pred = cnn_model.predict(x_test_np)

In [9]:
y_test_np = np.array(y_test).astype('int')
print(np.average(y_pred.flatten() - y_test_np))
print(y_pred.flatten())
print(y_test_np)
print(y_test_np-y_pred.flatten())

-168.14737590154013
[570.0415    391.2285    299.49963   537.54987   229.06223   271.21716
 253.16357     1.8840995 340.79242   634.80237   480.06262   348.58014
 273.40656   288.07565   400.26498   532.8608    250.59332   330.68503
 411.7417    298.6304    242.15988   242.38947    91.19758   396.48895
 269.41028   183.61203   294.36942     1.8840995 556.5113    430.4132   ]
[ 950  370  330  887  400  370  220  390  570 1200  300  300  140  480
  750  600  500  680  840  380  380  350  420  630  360  430  270  280
  700  420]
[ 379.95849609  -21.22848511   30.50036621  349.45013428  170.93777466
   98.78283691  -33.16357422  388.11590052  229.20758057  565.19763184
 -180.06262207  -48.58013916 -133.40655518  191.92434692  349.73501587
   67.13922119  249.40667725  349.31497192  428.25830078   81.36959839
  137.84011841  107.61053467  328.80242157  233.51104736   90.58972168
  246.38796997  -24.36941528  278.11590052  143.4887085   -10.41320801]


In [11]:
pip uninstall tensorflow

Uninstalling tensorflow-1.15.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/freeze_graph
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-1.15.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
    /usr/local/lib/python3.6/dist-packages/tensorflow_core/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-1.15.0


In [12]:
pip install tensorflow==1.12


     |████████████████████████████████| 83.1MB 118kB/s 
     |████████████████████████████████| 3.1MB 30.8MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [17]:
from tensorflow import lite

cnn_model.save('cnn_untrained_30ep_8b_adam_burgers.h5')
converter = lite.TFLiteConverter.from_keras_model_file('cnn_untrained_30ep_8b_adam_burgers.h5')
converter.allow_custom_ops = True
tfmodel = converter.convert()
open("model.tflite","wb").write(tfmodel)


INFO:tensorflow:Froze 28 variables.
INFO:tensorflow:Converted 28 variables to const ops.


3314092

In [0]:
#import pickle
#filename = 'cnn_untrained_30ep_8b_adam.sav'
#pickle.dump(cnn_model, open(filename, 'wb'))

In [0]:
#from flask import Flask, jsonify, request, render_template

In [0]:
#app = Flask(__name__)

In [0]:
#MODEL = pickle.load(open('cnn_untrained_30ep_8b_adam.sav', 'rb'))

In [0]:
#@app.route('/api', methods=['GET'])
#def api():
#    """Handle request and output model score in json format."""
   # Handle empty requests.
#    if not request.json:
#        return jsonify({'error': 'no request received'})
    # Parse request args into feature array for prediction.
 #   np_array_input = parse_args(request.json)
    # Predict on x_array and return JSON response.
#    estimate = int(MODEL.predict(np_array_input)[0])
#    return estimate

#def parse_args(json):
#    inp = json.get("file")
#    img = load_img(inp, target_size=(224,224,3))
#    return np.array(img)


In [0]:
#if __name__ == '__main__':
#    app.run(host='0.0.0.0', port=5000, debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
 * Restarting with stat


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
#exit

In [0]:
#f = "drive/My Drive/subway/bk_660.png"
#img = load_img(f, target_size=(224,224,3))
#test = np.array([np.array(img)])
#MODEL.predict(test)

array([[562.79755]], dtype=float32)